In [1]:
import pandas as pd
import numpy as np
import re

Load Dataset

In [2]:
df = pd.read_csv('../data/US_Accidents_March23_undersample.csv')
df.head(10)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-6719495,Source1,4,2020-12-12 21:30:00,2020-12-13 00:49:20,37.992617,-77.769182,37.993281,-77.767959,0.081,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-565753,Source2,1,2022-07-15 07:46:29,2022-07-15 08:35:22,35.158150,-80.734772,NaN,NaN,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-3426406,Source1,4,2016-12-15 10:42:57,2016-12-15 16:42:57,38.509280,-90.622916,38.507210,-90.623250,0.144,...,False,False,False,False,True,False,Day,Day,Day,Day
3,A-561207,Source2,1,2022-07-20 07:18:27,2022-07-20 08:18:16,35.084339,-82.468781,NaN,NaN,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-5422559,Source1,4,2022-11-04 02:22:00,2022-11-04 03:46:19,33.999802,-118.280993,33.999334,-118.280945,0.032,...,False,False,False,False,False,False,Night,Night,Night,Night
5,A-1024783,Source2,4,2021-05-30 22:46:17,2021-05-30 23:49:42,25.844481,-80.207092,NaN,NaN,1.750,...,False,False,False,False,False,False,Night,Night,Night,Night
6,A-3537389,Source1,4,2016-11-21 01:38:24,2016-11-21 07:38:24,36.104740,-115.295460,36.096760,-115.291200,0.600,...,False,False,False,False,False,False,Night,Night,Night,Night
7,A-7016796,Source1,1,2020-05-08 17:41:00,2020-05-08 20:00:58,35.600950,-82.360490,35.600950,-82.360490,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
8,A-7069748,Source1,3,2020-06-06 13:27:47,2020-06-06 14:17:45,40.823680,-73.877450,40.823680,-73.877450,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
9,A-2386148,Source2,3,2018-12-08 17:45:27,2018-12-08 18:28:42,39.642830,-75.803528,NaN,NaN,0.310,...,False,False,False,False,False,False,Night,Night,Night,Day


In [3]:
# number of duplicated data
df.duplicated().sum()

0

In [4]:
df.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [5]:
# drop columns that are not useful
not_useful = ['ID', 'Source', 'End_Time', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Weather_Timestamp']
df = df.drop(not_useful, axis=1)
df.columns

Index(['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Street', 'City',
       'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [6]:
categorical_columns = [
    'Severity', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
    'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump',
    'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
    'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
    'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'
]

numerical_columns = [
    'Start_Lat', 'Start_Lng', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
    'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)'
]

In [7]:
for i in categorical_columns:  
  print(i,df[i].unique().size)

Severity 4
Street 39440
City 7996
County 1503
State 49
Zipcode 46005
Country 1
Timezone 5
Airport_Code 1810
Wind_Direction 25
Weather_Condition 86
Amenity 2
Bump 2
Crossing 2
Give_Way 2
Junction 2
No_Exit 2
Railway 2
Roundabout 2
Station 2
Stop 2
Traffic_Calming 2
Traffic_Signal 2
Turning_Loop 1
Sunrise_Sunset 3
Civil_Twilight 3
Nautical_Twilight 3
Astronomical_Twilight 3


In [8]:
# drop country and turning loop because they have only one unique value
df = df.drop(['Country','Turning_Loop'], axis=1)

In [9]:
print("All wind Direction: ", df['Wind_Direction'].unique())

All wind Direction:  ['SSW' 'CALM' 'North' 'SW' 'NE' 'WNW' 'S' 'NW' 'SE' 'NNW' nan 'W' 'NNE'
 'West' 'South' 'ESE' 'Calm' 'E' 'East' 'WSW' 'VAR' 'SSE' 'N' 'ENE'
 'Variable']


In [10]:
# simplify wind directions
direction_mapping = {
    'Calm': 'CALM',
    'West': 'W', 'WSW': 'W', 'WNW': 'W',
    'South': 'S', 'SSW': 'S', 'SSE': 'S',
    'North': 'N', 'NNW': 'N', 'NNE': 'N',
    'East': 'E', 'ESE': 'E', 'ENE': 'E',
    'Variable': 'VAR'
}

# apply the mapping to the 'Wind_Direction' column
df['Wind_Direction'] = df['Wind_Direction'].map(direction_mapping).fillna(df['Wind_Direction'])
print("All wind directions after simplification:", df['Wind_Direction'].unique())

All wind directions after simplification: ['S' 'CALM' 'N' 'SW' 'NE' 'W' 'NW' 'SE' nan 'E' 'VAR']


In [11]:
# combine unique weather conditions into a single string, separating with '!'
weather_conditions_str = '!'.join(df['Weather_Condition'].dropna().unique().tolist())

# split the string using specified delimiters and filter out repetitive elements
unique_weather_conditions = np.unique(np.array(re.split(
    r"!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather_conditions_str))).tolist()
print("Weather Conditions:", unique_weather_conditions)

Weather Conditions: ['', 'Clear', 'Cloudy', 'Drizzle', 'Dust', 'Dust Whirlwinds', 'Fair', 'Fog', 'Hail', 'Haze', 'Heavy ', 'Heavy Drizzle', 'Heavy Rain', 'Heavy Sleet', 'Heavy Snow', 'Heavy T-Storm', 'Heavy Thunderstorms', 'Ice Pellets', 'Light ', 'Light Drizzle', 'Light Ice Pellets', 'Light Rain', 'Light Rain Shower', 'Light Rain Showers', 'Light Sleet', 'Light Snow', 'Light Snow Showers', 'Light Thunderstorms', 'Low Drifting Snow', 'Mist', 'N/A Precipitation', 'Overcast', 'Patches of Fog', 'Rain', 'Rain Showers', 'Sand', 'Scattered Clouds', 'Shallow Fog', 'Showers in the Vicinity', 'Sleet', 'Smoke', 'Snow', 'Squalls', 'T-Storm', 'Thunder', 'Thunder in the Vicinity', 'Thunderstorm', 'Thunderstorms', 'Widespread Dust', 'Windy', 'Wintry Mix']


In [12]:
# map weather conditions to columns
weather_conditions = {
    'Clear': 'Clear',
    'Cloud': 'Cloud|Overcast',
    'Rain': 'Rain|storm',
    'Heavy_Rain': 'Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms',
    'Snow': 'Snow|Sleet|Ice',
    'Heavy_Snow': 'Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls',
    'Fog': 'Fog'
}

# apply conditions and create new columns
for column, condition in weather_conditions.items():
    df[column] = df['Weather_Condition'].str.contains(condition, case=False, na=False)


In [13]:
weather_conditions = ['Clear', 'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog']

for condition in weather_conditions:
    null_condition = df['Weather_Condition'].isnull()
    df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
    df[condition] = df[condition].astype(bool)

df.loc[:, ['Weather_Condition'] + weather_conditions]

df = df.drop('Weather_Condition', axis=1)


/var/folders/dp/ptv4cbpd6fvf95gkd_vy0df00000gn/T/ipykernel_45362/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
/var/folders/dp/ptv4cbpd6fvf95gkd_vy0df00000gn/T/ipykernel_45362/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
/var/folders/dp/ptv4cbpd6fvf95gkd_vy0df00000gn/T/ipykernel_45362/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fu

In [14]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

df['Year'] = df['Start_Time'].dt.year
df['Month'] = df['Start_Time'].dt.month
df['Day'] = df['Start_Time'].dt.day
df['Hour'] = df['Start_Time'].dt.hour
df['Minute']=df['Hour']*60.0+df["Start_Time"].dt.minute

df.loc[:4,['Start_Time', 'Year', 'Month',  'Day', 'Hour', 'Minute']]
df = df.drop('Start_Time', axis=1)

In [15]:
missing_values = pd.DataFrame(df.isnull().sum(), columns=['Missing_Count']).reset_index()
missing_values.columns = ['Feature', 'Missing_Percent(%)']
missing_values['Missing_Percent(%)'] = missing_values['Missing_Percent(%)'] / df.shape[0] * 100
missing_values = missing_values[missing_values['Missing_Percent(%)'] > 0]
missing_values


,Feature,Missing_Percent(%)
3,Street,0.149185
4,City,0.002969
7,Zipcode,0.026720
8,Timezone,0.084612
9,Airport_Code,0.319894
10,Temperature(F),2.239260
11,Wind_Chill(F),24.901286
12,Humidity(%),2.394383
13,Pressure(in),1.880029
14,Visibility(mi),2.411454


In [16]:
# dropping wind chill because it will be hard to impute since it is missing 25% of the data
df = df.drop('Wind_Chill(F)', axis=1)

In [17]:
df['Precipitation_NA'] = 0
df.loc[df['Precipitation(in)'].isnull(), 'Precipitation_NA'] = 1
df['Precipitation(in)'] = df['Precipitation(in)'].fillna(df['Precipitation(in)'].median())
df.loc[:5, ['Precipitation(in)', 'Precipitation_NA']]

,Precipitation(in),Precipitation_NA
0,0.0,0
1,0.0,0
2,0.0,1
3,0.0,0
4,0.0,0
5,0.0,0


In [18]:
df = df.dropna(subset=['Street','City','Zipcode','Airport_Code',
                       'Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight', 'Wind_Direction'])

In [19]:
# impute missing values with median
for column in ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']:
    df[column] = df[column].fillna(df[column].median())

In [20]:
df.isnull().sum().sum()

0

In [21]:
df.columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'Street', 'City', 'County',
       'State', 'Zipcode', 'Timezone', 'Airport_Code', 'Temperature(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Clear',
       'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Precipitation_NA'],
      dtype='object')

In [23]:
# save the cleaned data
df.to_csv('../data/US_Accidents_March23_cleaned.csv', index=False)

# random sample of the cleaned data
df.sample(10000).to_csv('../data/US_Accidents_March23_cleaned_10000rows.csv', index=False)